# 1.cargamos el dataset de ventas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import pandas as pd
from scipy import stats
import re
from datetime import datetime
import sklearn
from sklearn import set_config
from sklearn import model_selection
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer


In [ ]:
df3 = pd.read_csv("/content/drive/MyDrive/datasets_TFM + diccionario/df_ventas.csv", index_col=0)#sergi dataset
#df3 = pd.read_csv("/content/drive/MyDrive/MASTER DATA SCIENCE 2023/TFM MASTER/EASY MONEY/df_ventas.csv", index_col=0)#jaume dataset

In [ ]:
df3.info(verbose = False)
# Este dataframe tiene 2419150  filas y 34 columnas
# las columnas tienen datos de tipo float64(5) - int64(3) - object(11) - datatime(3)-bool(16)
# no hay nulos

<class 'pandas.core.frame.DataFrame'>
Index: 1653628 entries, 0 to 5962923
Columns: 33 entries, pk_cid to SalaryRange
dtypes: bool(16), float64(4), int64(2), object(11)
memory usage: 252.3+ MB


In [ ]:
df3.isna().sum()

pk_cid                0
pk_partition          0
short_term_deposit    0
loans                 0
mortgage              0
funds                 0
securities            0
long_term_deposit     0
credit_card           0
payroll               0
pension_plan          0
payroll_account       0
emc_account           0
debit_card            0
em_account_p          0
em_acount             0
entry_date            0
entry_channel         0
active_customer       0
segment               0
country_id            0
region_code           0
gender                0
age                   0
deceased              0
pk_sale               0
month_sale            0
product_ID            0
net_margin            0
product_desc          0
family_product        0
Provincia             0
SalaryRange           0
dtype: int64

In [ ]:
#Estas son las columnas que tenemos en nuestro dataset
print(df3.columns.tolist())

['pk_cid', 'pk_partition', 'short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 'credit_card', 'payroll', 'pension_plan', 'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount', 'entry_date', 'entry_channel', 'active_customer', 'segment', 'country_id', 'region_code', 'gender', 'age', 'deceased', 'pk_sale', 'month_sale', 'product_ID', 'net_margin', 'product_desc', 'family_product', 'Provincia', 'SalaryRange']


In [ ]:
#visualizamos la tabla
df3.head().T

,0,12,32,36,41
pk_cid,1375586,1050623,1050602,1050580,1050676
pk_partition,2018-01-01,2018-01-01,2018-01-01,2018-01-01,2018-01-01
short_term_deposit,False,False,False,False,False
loans,False,False,False,False,False
mortgage,False,False,False,False,False
funds,False,False,False,False,False
securities,False,False,False,False,False
long_term_deposit,False,False,False,False,False
credit_card,False,False,False,False,False
payroll,False,False,False,False,False


#2.Definimos el TARGET

In [ ]:
TARGET = 'product_desc'

##3.funciones a utilizar

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Asumimos que TARGET está definida en alguna parte de tu código, por ejemplo:
# TARGET = 'nombre_de_la_columna_objetivo'

def analizar_columna(df, columna, valor_referencia=0.5):
    global TARGET  # Utilizamos la variable TARGET definida globalmente
    if columna not in df.columns or TARGET not in df.columns:
        print(f"Verifique que tanto '{columna}' como '{TARGET}' existen en el DataFrame.")
        return

    # 1. Conteo de valores únicos
    print(f"\nConteo de valores únicos en '{columna}':\n{df[columna].value_counts()}\n")

    # 2. Estadísticas descriptivas
    print(f"Estadísticas descriptivas de '{columna}':\n{df[columna].describe()}\n")

    # Verifica si la columna es de tipo numérico
    if df[columna].dtype in ['int64', 'float64']:
        # 3. Valor máximo y mínimo
        max_value = df[columna].max()
        min_value = df[columna].min()
        print(f"Valor máximo en '{columna}': {max_value}")
        print(f"Valor mínimo en '{columna}': {min_value}")

    # 4. Histograma de la columna
    plt.figure(figsize=(10, 6))
    sns.histplot(df[columna], bins=25, kde=False, color='green')
    plt.title(f"Histograma de '{columna}'")
    plt.xlabel(columna)
    plt.ylabel("Frecuencia")
    plt.xticks(rotation=45, ha='right')
    plt.grid(False)
    plt.show()

    # Análisis de relación con la columna objetivo
    if TARGET in df.columns:
        relacion = df.groupby(columna)[TARGET].mean().sort_values()
        print(f"\nMedia de '{TARGET}' por valor de '{columna}':\n{relacion}\n")

        # Identifica y muestra los valores cuya media se desvíe significativamente de un valor de referencia
        desviacion_significativa = relacion[(relacion < valor_referencia * 0.8) | (relacion > valor_referencia * 1.2)]
        print(f"Valores en '{columna}' con una media de '{TARGET}' significativamente distinta de {valor_referencia}:\n{desviacion_significativa}\n")


## 4.Analisis

In [ ]:
list_data = ["pk_partition", "entry_date", "month_sale"]

# Convertir las columnas especificadas a datetime
for column in list_data:
    df3[column] = pd.to_datetime(df3[column])

In [ ]:
to_object = ['product_ID', 'region_code', 'pk_sale', 'product_ID']
to_bool = ['active_customer']

In [ ]:
for i in to_object:
  df3[i] = df3[i].astype(object)

In [ ]:
df3[to_bool] = df3[to_bool].astype(bool)

In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1653628 entries, 0 to 5962923
Data columns (total 33 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   pk_cid              1653628 non-null  int64         
 1   pk_partition        1653628 non-null  datetime64[ns]
 2   short_term_deposit  1653628 non-null  bool          
 3   loans               1653628 non-null  bool          
 4   mortgage            1653628 non-null  bool          
 5   funds               1653628 non-null  bool          
 6   securities          1653628 non-null  bool          
 7   long_term_deposit   1653628 non-null  bool          
 8   credit_card         1653628 non-null  bool          
 9   payroll             1653628 non-null  bool          
 10  pension_plan        1653628 non-null  bool          
 11  payroll_account     1653628 non-null  bool          
 12  emc_account         1653628 non-null  bool          
 13  debit_card       

## 5.Calculamos varianza

In [ ]:
def calcular_varianza(df):
    # Seleccionar solo las columnas numéricas, excluyendo datetime y object
    columnas_numericas = df.select_dtypes(exclude=['object', 'datetime64[ns]'])

    # Calcular la varianza para cada columna numérica
    varianzas = columnas_numericas.var()
    return varianzas

# Usar la función con el DataFrame df3
varianza_resultado = calcular_varianza(df3)

# Imprimir las varianzas calculadas
print(varianza_resultado)


pk_cid                3.065781e+10
short_term_deposit    8.832503e-03
loans                 1.529736e-04
mortgage              1.033984e-04
funds                 7.782442e-03
securities            7.774108e-03
long_term_deposit     3.888871e-02
credit_card           2.653526e-02
payroll               8.195533e-02
pension_plan          8.560813e-02
payroll_account       1.279458e-01
emc_account           9.035409e-02
debit_card            1.631054e-01
em_account_p          1.028033e-05
em_acount             1.880037e-01
active_customer       2.075749e-01
gender                2.491131e-01
age                   1.818881e+02
net_margin            3.517777e+05
dtype: float64


In [ ]:
df3['long_term_deposit'].value_counts()

long_term_deposit
False    1586604
True       67024
Name: count, dtype: int64

In [ ]:
def remove_low_variance_columns(df, threshold):
    low_variance_cols = []
    for col in df.columns:
        if np.var(df[col]) < threshold:
            low_variance_cols.append(col)
    return df.drop(low_variance_cols, axis=1)

In [ ]:
remove_low_variance_columns

<function __main__.remove_low_variance_columns(df, threshold)>

## 6.Procedemos a analizar las columnas y ver y corregir outliers

In [ ]:
analizar_columna(df3,"pk_partition")#Identificador de Partición. Mensualmente se ingesta en la tabla una partición con el estado de la base de clientes.

In [ ]:
analizar_columna(df3,"short_term_deposit")#Tenencia de Short-term deposits

In [ ]:
analizar_columna(df3,"loans")#elinamos por poca relevancia

In [ ]:
analizar_columna(df3,"mortgage")#un tipo de hipoteca(valorar si borrar o no )

In [ ]:
#df3.drop(columns=["mortgage","loans"], inplace=True)#aqui eliminamos las columnas por poca relevancia

In [ ]:
df3.isna().sum()

In [ ]:
analizar_columna(df3,"funds")#fondos de inversion

In [ ]:
analizar_columna(df3,"securities")#tenencia de valores

In [ ]:
analizar_columna(df3,"long_term_deposit")#depositos a largo plazo

In [ ]:
analizar_columna(df3,"credit_card")#tarjeta de credito

In [ ]:
analizar_columna(df3,"payroll")#tenencia de domiciliaciones

In [ ]:
analizar_columna(df3,"pension_plan")#tenencia de plan de pensiones

In [ ]:
analizar_columna(df3,"payroll_account")#cuenta bonificada por domiciliaciones

In [ ]:
analizar_columna(df3,"emc_account")#cuentas de easy money crypto

In [ ]:
analizar_columna(df3,"debit_card")#tarjeta de debito

In [ ]:
analizar_columna(df3,"em_account_p")#para eliminar poca varianza

In [ ]:
df3.drop(columns=["em_account_p"], inplace=True)

In [ ]:
analizar_columna(df3,"em_acount")#tenencia de cuenta de easymoney

In [ ]:
analizar_columna(df3,"entry_date")#fecha de entrada

In [ ]:
analizar_columna(df3,"entry_channel")#canal de entrada

In [ ]:
valores_a_mantener =  ["KHQ", "KHE", "KFC", "KHK", "KHM", "KAT", "KHN", "RED", "KHL", "KFA"]

# Reemplaza todos los otros valores en 'entry_channel' con "Others"
df3.loc[~df3['entry_channel'].isin(valores_a_mantener), 'entry_channel'] = 'Others'

# Verifica los cambios
print(df3['entry_channel'].value_counts())

In [ ]:
analizar_columna(df3,"active_customer")#Indicador de actividad del cliente en la app EasyMoney

In [ ]:
df3[[ 'active_customer', 'month_sale']][
    df3['active_customer'].notna() & df3['month_sale'].notna()]

In [ ]:
analizar_columna(df3,"segment")#segmento comercial del cliente

In [ ]:
analizar_columna(df3,"country_id")#pais de residencia del cliente

In [ ]:
analizar_columna(df3,"region_code")#region del cliente (solo ESP)

In [ ]:
df3.isna().sum()

In [ ]:
plt.figure(figsize=(10,10))
sns.scatterplot(x="SalaryRange", y="Provincia", data=df3)
plt.title('Relación entre Salario y region')
plt.show()

In [ ]:
df3['gender'] = df3['gender'].replace({True: 'M', False: 'H'})

In [ ]:
analizar_columna(df3,"gender")#pone H y V

In [ ]:
analizar_columna(df3,"age")

In [ ]:
analizar_columna(df3,"deceased")#si esta muerto o vivo

In [ ]:
df3.drop(columns=["deceased"], inplace=True)

In [ ]:
analizar_columna(df3,"SalaryRange")#de la unidad familiar (padre madre )

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x="SalaryRange", y="age", data=df3)
plt.title('Relación entre Salario y Edad')
plt.show()

In [ ]:
analizar_columna(df3,"age")#edad del cliente

In [ ]:
analizar_columna(df3,"salary")

In [ ]:
analizar_columna(df3,"pk_sale")

In [ ]:
analizar_columna(df3,"month_sale")

In [ ]:
analizar_columna(df3,"product_ID")

In [ ]:
analizar_columna(df3,"net_margin")

In [ ]:
analizar_columna(df3,"product_desc")

In [ ]:
analizar_columna(df3,"family_product")

In [ ]:
df3.info()

In [ ]:
df3['active_customer'].value_counts()

In [ ]:
df3.select_dtypes(exclude=object).head()

In [ ]:
df3.select_dtypes(include=object).head()

In [ ]:
df3.info()

In [ ]:
# Convertir todas las columnas booleanas a enteros (1 y 0)
df3 = df3.apply(lambda x: x.astype(int) if x.dtype == bool else x)

print(df3)

In [ ]:
historial_compras = df3.groupby(['pk_cid', 'product_ID']).size().reset_index(name='frecuencia_compra')
# Fusionar la tabla de frecuencia de compras con el DataFrame original
df3 = pd.merge(df3, historial_compras, on=['pk_cid','product_ID'], how='left')

In [ ]:
df3.head().T

In [ ]:
# Función para determinar el grupo de edad
def age_group(age):
    if age < 25:
        return 'Joven'
    elif age <= 45:
        return 'Adulto'
    else:
        return 'Senior'

# Función para determinar el grupo principal de productos
def product_group(row):
    if row['loans'] == 1 or row['mortgage'] == 1:
        return 'Prestamos'
    elif row['funds'] == 1 or row['securities'] == 1 or row['long_term_deposit'] == 1:
        return 'Inversiones'
    elif row['payroll_account'] == 1 or row['emc_account'] == 1 or row['em_acount'] == 1:
        return 'Cuentas'
    else:
        return 'Otros'

# Aplicar las funciones y crear la columna "Product_Age"
df3['Age_Group'] = df3['age'].apply(age_group)
df3['Product_Group'] = df3.apply(product_group, axis=1)
df3['Product_Age'] = df3['Age_Group'] + "_" + df3['Product_Group']

In [ ]:
df3.head().T

In [ ]:
df3["Age_Group"].value_counts()

In [ ]:
df3["Product_Group"].value_counts()

In [ ]:
file_path_csv_gzip = 'df3_powerbi_def.csv.gz'
df3.to_csv(file_path_csv_gzip, index=True, sep=',', compression='gzip')

##Segmentación

In [ ]:
# Realizamos la segmentación por tipo de producto
segmentacion_por_producto = df3.groupby('product_desc').size()
print(segmentacion_por_producto)

In [ ]:
#CONCLUSIONES:

In [ ]:
# Realizamos la segmentación por provincia
segmentacion_por_provincia = df3.groupby('Provincia').size()
segmentacion_ordenada = segmentacion_por_provincia.sort_values(ascending=False)
print(segmentacion_ordenada)


In [ ]:
#CONCLUSIONES:

In [ ]:
# Agrupamos por Provincia y product_desc, y contamos las ocurrencias
productos_por_provincia = df3.groupby(['Provincia', 'product_desc']).size()
# Convertimos el resultado en un DataFrame para mejor manejo
productos_por_provincia_df = productos_por_provincia.reset_index(name='count')
# Ordenamos dentro de cada provincia por count de manera descendente para ver los más populares al principio
productos_por_provincia_df.sort_values(by=['Provincia', 'count'], ascending=[True, False], inplace=True)
for provincia in productos_por_provincia_df['Provincia'].unique():
    print(f"Productos más populares en {provincia}:")
    print(productos_por_provincia_df[productos_por_provincia_df['Provincia'] == provincia].head())  # Mostrar los 5 productos más populares por provincia
    print("\n")


In [ ]:
#CONCLUSIONES:

In [ ]:
# Agrupar por cliente y producto para ver la frecuencia de compra de cada producto
historial_compras = df3.groupby(['pk_cid', 'product_ID']).size().reset_index(name='frecuencia_compra')

# También podemos observar la distribución temporal de las compras por cliente
compras_por_mes = df3.groupby(['pk_cid', 'month_sale']).size().reset_index(name='compras_mes')

# Para visualizar mejor, se podría ordenar y limitar a los clientes con más compras o más lealtad a cierto producto
historial_compras.sort_values(by='frecuencia_compra', ascending=False, inplace=True)
compras_por_mes.sort_values(by='compras_mes', ascending=False, inplace=True)

print(historial_compras.sample(30))  # Mostrar los clientes con mayor frecuencia de compra de un producto específico
print(compras_por_mes.sample(30))    # Mostrar los clientes con más compras en un mes específico


In [ ]:
#CONCLUSIONES:

In [ ]:
# Clasificación de clientes por margen neto
# Se podría definir un umbral para clasificar a los clientes en 'alto valor' y 'bajo valor' basado en el margen neto
umbral_alto_valor = df3['net_margin'].quantile(0.70)  # por ejemplo, el percentil 75 como umbral de alto valor

# Creamos una nueva columna para clasificar a los clientes
df3['clasificacion_valor'] = df3['net_margin'].apply(lambda x: 'Alto Valor' if x >= umbral_alto_valor else 'Bajo Valor')

# Contamos cuántos clientes hay en cada clasificación
clasificacion_valor_counts = df3['clasificacion_valor'].value_counts()

print(clasificacion_valor_counts)


In [ ]:
df3.info()

## 7.Clustering

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Lista de características usadas para el clustering
features = ['age', 'net_margin', 'short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 'credit_card', 'payroll','salary']

# Selecciona solo las columnas de características del DataFrame
df_features = df3[features]

# Escala las características para que tengan media=0 y desviación estándar=1
scaler = StandardScaler()
df_features_scaled = pd.DataFrame(scaler.fit_transform(df_features), columns=features)

# Definir y ajustar el modelo K-Means
# El número de clusters (n_clusters) puede ser elegido basado en tu conocimiento del dominio o técnicas como el método del codo
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(df_features_scaled) # Fit the model on scaled data

# Calcula el promedio general para las características escaladas
general_means = df_features_scaled.mean()

# Obtiene los centros de los clusters desde el modelo KMeans
cluster_centers = kmeans.cluster_centers_

# Convierte los centros de los clusters a un DataFrame para facilitar su manipulación
df_cluster_centers = pd.DataFrame(cluster_centers, columns=features)

# Compara los centros de los clusters con el promedio general
for i, center in df_cluster_centers.iterrows():
    print(f"Cluster {i} deviations from general mean:")
    print(center - general_means)
    print("\n")


In [ ]:

df3['Cluster'] = kmeans.labels_

# 2. Análisis Descriptivo por Cluster
# Calcularemos estadísticas descriptivas para la columna de edad agrupadas por cluster
age_descriptive_stats = df3.groupby('Cluster')['age'].describe()

print(age_descriptive_stats)


Cluster 0

* Perfil: Ligeramente mayor edad que el promedio.
* Finanzas: Bajo margen neto, menor uso de fondos y depósitos a largo plazo.
* Productos Usados: Alta utilización de tarjetas de crédito y nómina, además de una notable inclinación hacia valores mobiliarios.

Este cluster podría representar a individuos en etapas estables de sus vidas,
activos en el uso de productos bancarios, con un enfoque en la inversión y el crédito.

Cluster 1


* Perfil: Edad ligeramente inferior al promedio.
* Finanzas: Bajo margen neto, baja participación en todos los productos financieros.
* Productos Usados: Menor uso de depósitos, fondos, tarjetas de crédito y productos de nómina.

Este cluster incluye probablemente a clientes más jóvenes o con menos recursos, posiblemente menos activos en banca e inversión, quizás debido a una menor capacidad financiera.

Cluster 2
* Perfil: Mayor edad y margen neto significativamente alto.
* Finanzas: Uso extremadamente alto de depósitos a corto plazo.
* Productos Usados: Bajo uso de la mayoría de los otros productos financieros.

parecen ser individuos maduros y financieramente seguros, enfocados en inversiones a corto plazo, posiblemente debido a una estrategia de gestión de riesgos o necesidades de liquidez inmediata.

Cluster 3

* Perfil: La mayor edad y margen neto entre todos los clusters.
* Finanzas: Uso significativo de fondos y el mayor uso de depósitos a largo plazo.
* Productos Usados: Alguno uso de crédito y valores mobiliarios.

 representa probablemente a individuos muy maduros y acaudalados, con un fuerte enfoque en la acumulación de riqueza y preparación para la jubilación, utilizando una amplia gama de productos de inversión.

Cluster 4

* Perfil: Ligeramente mayor edad que el promedio.
* Finanzas: El mayor margen neto de todos los clusters.
* Productos Usados: Uso excepcionalmente alto de préstamos, muy alto uso de tarjetas de crédito y una participación moderada en fondos.

Clientes en este cluster son probablemente individuos con necesidades financieras sustanciales, activos en el uso de productos de crédito y préstamos, lo que puede reflejar una etapa de inversión significativa en negocios o propiedades personales.

In [ ]:
ficha_df = df3

In [ ]:
(
    df3
    .groupby("Cluster")
    .describe()#mean
    .T
    .style.background_gradient(cmap = 'Blues', axis = 1)
)

##Modelo de classificacion

In [ ]:
df3.head().T

,0,12,32,36,41
pk_cid,1375586,1050623,1050602,1050580,1050676
pk_partition,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00
short_term_deposit,False,False,False,False,False
loans,False,False,False,False,False
mortgage,False,False,False,False,False
funds,False,False,False,False,False
securities,False,False,False,False,False
long_term_deposit,False,False,False,False,False
credit_card,False,False,False,False,False
payroll,False,False,False,False,False


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1653628 entries, 0 to 5962923
Data columns (total 33 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   pk_cid              1653628 non-null  int64         
 1   pk_partition        1653628 non-null  datetime64[ns]
 2   short_term_deposit  1653628 non-null  bool          
 3   loans               1653628 non-null  bool          
 4   mortgage            1653628 non-null  bool          
 5   funds               1653628 non-null  bool          
 6   securities          1653628 non-null  bool          
 7   long_term_deposit   1653628 non-null  bool          
 8   credit_card         1653628 non-null  bool          
 9   payroll             1653628 non-null  bool          
 10  pension_plan        1653628 non-null  bool          
 11  payroll_account     1653628 non-null  bool          
 12  emc_account         1653628 non-null  bool          
 13  debit_card       

### 4. Modelling

In [ ]:
df4 =df3.copy()

In [ ]:
df4['part_month'] = df4['pk_partition'].dt.month
df4['part_year'] = df4['pk_partition'].dt.year
#df4['part_day'] = df4['pk_partition'].dt.day
df4['part_weekday'] = df4['pk_partition'].dt.weekday

df4['month_month'] = df4['month_sale'].dt.month
df4['month_year'] = df4['month_sale'].dt.year
#df4['month_day'] = df4['month_sale'].dt.day
df4['month_weekday'] = df4['month_sale'].dt.weekday

df4['entry_month'] = df4['entry_date'].dt.month
df4['entry_year'] = df4['entry_date'].dt.year
#df4['entry_day'] = df4['entry_date'].dt.day
df4['entry_weekday'] = df4['entry_date'].dt.weekday

#df4.drop('pk_partition', axis=1, inplace=True)
df4.drop('month_sale', axis=1, inplace=True)
df4.drop('entry_date', axis=1, inplace=True)

In [ ]:
df4.columns

Index(['pk_cid', 'pk_partition', 'short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'credit_card', 'payroll',
       'pension_plan', 'payroll_account', 'emc_account', 'debit_card',
       'em_account_p', 'em_acount', 'entry_channel', 'active_customer',
       'segment', 'country_id', 'region_code', 'gender', 'age', 'deceased',
       'pk_sale', 'product_ID', 'net_margin', 'product_desc', 'family_product',
       'Provincia', 'SalaryRange', 'part_month', 'part_year', 'part_weekday',
       'month_month', 'month_year', 'month_weekday', 'entry_month',
       'entry_year', 'entry_weekday'],
      dtype='object')

In [ ]:
drop_columns = ['short_term_deposit', 'loans', 'mortgage', 'funds',
       'securities', 'long_term_deposit', 'credit_card', 'payroll',
       'pension_plan', 'payroll_account', 'emc_account', 'debit_card',
       'em_account_p', 'em_acount']

In [ ]:
df4['pk_cid'] = df4['pk_cid'].astype(object)

In [ ]:
boolean_columns = df4.select_dtypes(include='bool').columns
df4[boolean_columns] = df4[boolean_columns].astype(int)

In [ ]:
for i in df4['product_desc'].unique():
  print(i)

debit_card
payroll
securities
em_acount
pension_plan
emc_account
credit_card
funds
long_term_deposit
payroll_account
mortgage
short_term_deposit
loans


In [ ]:
df4.drop(columns = drop_columns, inplace=True)

In [ ]:
# 1. Cantidad total de productos comprados por cada usuario
total_products = df4.groupby('pk_cid')['product_desc'].count().reset_index()
total_products.rename(columns={'product_desc': 'total_products'}, inplace=True)

In [ ]:
debit_card = df4[df4['product_desc'] == 'debit_card'].groupby('pk_cid').size().reset_index(name='debit_card')
payroll = df4[df4['product_desc'] == 'payroll'].groupby('pk_cid').size().reset_index(name='payroll')
securities = df4[df4['product_desc'] == 'securities'].groupby('pk_cid').size().reset_index(name='securities')
em_acount = df4[df4['product_desc'] == 'em_acount'].groupby('pk_cid').size().reset_index(name='em_acount')
credit_card = df4[df4['product_desc'] == 'credit_card'].groupby('pk_cid').size().reset_index(name='credit_card')
funds = df4[df4['product_desc'] == 'funds'].groupby('pk_cid').size().reset_index(name='funds')
long_term_deposit = df4[df4['product_desc'] == 'long_term_deposit'].groupby('pk_cid').size().reset_index(name='long_term_deposit')
payroll_account = df4[df4['product_desc'] == 'payroll_account'].groupby('pk_cid').size().reset_index(name='payroll_account')
mortgage = df4[df4['product_desc'] == 'mortgage'].groupby('pk_cid').size().reset_index(name='mortgage')
short_term_deposit = df4[df4['product_desc'] == 'short_term_deposit'].groupby('pk_cid').size().reset_index(name='short_term_deposit')
loans = df4[df4['product_desc'] == 'loans'].groupby('pk_cid').size().reset_index(name='loans')

In [ ]:
loans

,pk_cid,loans
0,808433,17
1,1008258,17
2,1031343,17
3,1075139,17
4,1075423,14
5,1090866,17
6,1126796,17
7,1128184,17
8,1134495,17
9,1134687,17


In [ ]:
last_purchase_date = df4.groupby('pk_cid')['pk_partition'].max().reset_index()
last_purchase_date.rename(columns={'pk_partition': 'last_purchase_date'}, inplace=True)

In [ ]:
purchase_dates = df4.groupby('pk_cid')['pk_partition'].apply(lambda x: x.diff().mean()).reset_index(name='purchase_time_variance')

In [ ]:
total_margin = df4.groupby('pk_cid')['net_margin'].sum().reset_index()
total_margin.rename(columns={'net_margin': 'total_margin'}, inplace=True)

In [ ]:
features_df = total_products.merge(debit_card, on='pk_cid', how='left')
features_df = features_df.merge(payroll, on='pk_cid', how='left')
features_df = features_df.merge(securities, on='pk_cid', how='left')
features_df = features_df.merge(em_acount, on='pk_cid', how='left')
features_df = features_df.merge(credit_card, on='pk_cid', how='left')
features_df = features_df.merge(funds, on='pk_cid', how='left')
features_df = features_df.merge(long_term_deposit, on='pk_cid', how='left')
features_df = features_df.merge(payroll_account, on='pk_cid', how='left')
features_df = features_df.merge(mortgage, on='pk_cid', how='left')
features_df = features_df.merge(short_term_deposit, on='pk_cid', how='left')
features_df = features_df.merge(loans, on='pk_cid', how='left')
features_df = features_df.merge(last_purchase_date, on='pk_cid', how='left')
features_df = features_df.merge(purchase_dates, on='pk_cid', how='left')
features_df = features_df.merge(total_margin, on='pk_cid', how='left')
features_df = df4.merge(features_df, on='pk_cid', how='left')

# Mostrar las primeras filas del DataFrame resultante
print(features_df.head())

    pk_cid pk_partition entry_channel  active_customer             segment  \
0  1375586   2018-01-01           KHL                1   02 - PARTICULARES   
1  1050623   2018-01-01           KHE                1  03 - UNIVERSITARIO   
2  1050602   2018-01-01           KHE                1  03 - UNIVERSITARIO   
3  1050580   2018-01-01           KHE                1  03 - UNIVERSITARIO   
4  1050676   2018-01-01           KFC                0   02 - PARTICULARES   

  country_id region_code  gender  age deceased  ... credit_card funds  \
0         ES        29.0       1   35        N  ...         NaN   NaN   
1         ES        49.0       1   23        N  ...         NaN   NaN   
2         ES        37.0       0   23        N  ...         NaN   NaN   
3         ES        27.0       1   23        N  ...         NaN   NaN   
4         ES        45.0       0   31        N  ...         NaN   NaN   

   long_term_deposit payroll_account mortgage short_term_deposit loans  \
0                N

In [ ]:
features_df = features_df.groupby('pk_cid').apply(lambda group: group.fillna(method='bfill').fillna(method='ffill'))

In [ ]:
features_df.isna().sum()

pk_cid                          0
pk_partition                    0
entry_channel                   0
active_customer                 0
segment                         0
country_id                      0
region_code                     0
gender                          0
age                             0
deceased                        0
pk_sale                         0
product_ID                      0
net_margin                      0
product_desc                    0
family_product                  0
Provincia                       0
SalaryRange                     0
part_month                      0
part_year                       0
part_weekday                    0
month_month                     0
month_year                      0
month_weekday                   0
entry_month                     0
entry_year                      0
entry_weekday                   0
total_products                  0
debit_card                1303797
payroll                   1416595
securities    

In [ ]:
features_df

pk_cid pk_partition entry_channel  active_customer  \
pk_cid                                                                 
15891   282174     15891   2018-07-01           KAT                1   
        437434     15891   2018-08-01           KAT                0   
16203   828563     16203   2018-12-01           KAT                0   
        1039916    16203   2019-01-01           KAT                1   
        1189603    16203   2019-02-01           KAT                1   
...                  ...          ...           ...              ...   
1553456 1565842  1553456   2019-05-01        UKNOWN                1   
1553541 1565839  1553541   2019-05-01        UKNOWN                1   
1553559 1565846  1553559   2019-05-01        UKNOWN                1   
1553565 1565845  1553565   2019-05-01        UKNOWN                0   
1553571 1565844  1553571   2019-05-01        UKNOWN                1   

                           segment country_id  region_code  gender  age  \
pk_cid                                                                    
15891   282174   04 - SINCATEGORIA         ES         28.0       1   59   
        437434   02 - PARTICULARES         ES         28.0       1   59   
16203   828563   04 - SINCATEGORIA         ES          8.0       0   70   
        1039916  02 - PARTICULARES         ES          8.0       0   70   
        1189603           01 - TOP         ES          8.0       0   70   
...                            ...        ...          ...     ...  ...   
1553456 1565842  04 - SINCATEGORIA         ES         28.0       1   32   
1553541 1565839  04 - SINCATEGORIA         ES         49.0       0   54   
1553559 1565846  04 - SINCATEGORIA         ES          8.0       1   43   
1553565 1565845  04 - SINCATEGORIA         ES         45.0       1   68   
1553571 1565844  04 - SINCATEGORIA         ES         15.0       0   30   

                deceased  ...  credit_card  funds  long_term_deposit  \
pk_cid                    ...                                          
15891   282174         N  ...          NaN    NaN                NaN   
        437434         N  ...          NaN    NaN                NaN   
16203   828563         N  ...          NaN    NaN                NaN   
        1039916        N  ...          NaN    NaN                NaN   
        1189603        N  ...          NaN    NaN                NaN   
...                  ...  ...          ...    ...                ...   
1553456 1565842        N  ...          NaN    NaN                NaN   
1553541 1565839        N  ...          NaN    NaN                NaN   
1553559 1565846        N  ...          NaN    NaN                NaN   
1553565 1565845        N  ...          NaN    NaN                NaN   
1553571 1565844        N  ...          NaN    NaN                NaN   

                payroll_account mortgage short_term_deposit loans  \
pk_cid                                                              
15891   282174              NaN      NaN                NaN   NaN   
        437434              NaN      NaN                NaN   NaN   
16203   828563              NaN      NaN                NaN   NaN   
        1039916             NaN      NaN                NaN   NaN   
        1189603             NaN      NaN                NaN   NaN   
...                         ...      ...                ...   ...   
1553456 1565842             NaN      NaN                NaN   NaN   
1553541 1565839             NaN      NaN                NaN   NaN   
1553559 1565846             NaN      NaN                NaN   NaN   
1553565 1565845             NaN      NaN                NaN   NaN   
1553571 1565844             NaN      NaN                NaN   NaN   

                 last_purchase_date  purchase_time_variance  total_margin  
pk_cid                                                                     
15891   282174           2018-08-01        31 days 00:00:00         132.8  
        437434           2018-08-01       

In [ ]:
features_df.fillna(0,inplace=True)

In [ ]:
features_df.reset_index(drop=True, inplace=True)

In [ ]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1653628 entries, 0 to 1653627
Data columns (total 41 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   pk_cid                  1653628 non-null  int64         
 1   pk_partition            1653628 non-null  datetime64[ns]
 2   entry_channel           1653628 non-null  object        
 3   active_customer         1653628 non-null  int64         
 4   segment                 1653628 non-null  object        
 5   country_id              1653628 non-null  object        
 6   region_code             1653628 non-null  float64       
 7   gender                  1653628 non-null  int64         
 8   age                     1653628 non-null  int64         
 9   deceased                1653628 non-null  object        
 10  pk_sale                 1653628 non-null  float64       
 11  product_ID              1653628 non-null  float64       
 12  net_margin    

In [ ]:
rows_with_value = features_df[features_df.eq('KAT').any(axis=1)]

# Mostrar las filas que contienen el valor 'KAT'
print(rows_with_value)

          pk_cid pk_partition entry_channel  active_customer  \
0          15891   2018-07-01           KAT                1   
1          15891   2018-08-01           KAT                0   
2          16203   2018-12-01           KAT                0   
3          16203   2019-01-01           KAT                1   
4          16203   2019-02-01           KAT                1   
...          ...          ...           ...              ...   
1627394  1530982   2019-04-01           KAT                1   
1627395  1530982   2019-05-01           KAT                1   
1628724  1531668   2019-03-01           KAT                1   
1634030  1534308   2019-04-01           KAT                1   
1634031  1534308   2019-05-01           KAT                1   

                   segment country_id  region_code  gender  age deceased  ...  \
0        04 - SINCATEGORIA         ES         28.0       1   59        N  ...   
1        02 - PARTICULARES         ES         28.0       1   59      

In [ ]:
# Obtener las columnas numéricas del DataFrame
columnas_numericas = features_df.select_dtypes(include='number').columns

# Inicializar una lista para almacenar las columnas que cumplen con el criterio
columnas_deseadas = []

# Iterar sobre las columnas numéricas y verificar si no contienen solo los valores 0 y 1
for columna in columnas_numericas:
    if features_df[columna].nunique() > 2:
        columnas_deseadas.append(columna)

# Verificar las columnas seleccionadas
print("Columnas numéricas que no contienen exclusivamente los valores 0 y 1:")
print(columnas_deseadas)

Columnas numéricas que no contienen exclusivamente los valores 0 y 1:
['pk_cid', 'region_code', 'age', 'pk_sale', 'product_ID', 'net_margin', 'part_month', 'part_weekday', 'month_month', 'month_weekday', 'entry_month', 'entry_year', 'entry_weekday', 'total_products', 'debit_card', 'payroll', 'securities', 'em_acount', 'credit_card', 'funds', 'long_term_deposit', 'payroll_account', 'mortgage', 'short_term_deposit', 'loans', 'total_margin']


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoded_df = pd.get_dummies(features_df.select_dtypes(include='object'), prefix='encoded')

# Concatenar el DataFrame original con las nuevas columnas codificadas
features_df = pd.concat([features_df, encoded_df], axis=1)

In [ ]:
features_df

,pk_cid,pk_partition,entry_channel,active_customer,segment,country_id,region_code,gender,age,deceased,...,encoded_36 days 09:36:00,encoded_92 days 00:00:00,encoded_90 days 12:00:00,encoded_45 days 06:00:00,encoded_50 days 08:00:00,encoded_151 days 00:00:00,encoded_40 days 00:00:00,encoded_120 days 00:00:00,encoded_28 days 00:00:00,encoded_44 days 12:00:00
0,15891,2018-07-01,KAT,1,04 - SINCATEGORIA,ES,28.0,1,59,N,...,False,False,False,False,False,False,False,False,False,False
1,15891,2018-08-01,KAT,0,02 - PARTICULARES,ES,28.0,1,59,N,...,False,False,False,False,False,False,False,False,False,False
2,16203,2018-12-01,KAT,0,04 - SINCATEGORIA,ES,8.0,0,70,N,...,False,False,False,False,False,False,False,False,False,False
3,16203,2019-01-01,KAT,1,02 - PARTICULARES,ES,8.0,0,70,N,...,False,False,False,False,False,False,False,False,False,False
4,16203,2019-02-01,KAT,1,01 - TOP,ES,8.0,0,70,N,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1653623,1553456,2019-05-01,UKNOWN,1,04 - SINCATEGORIA,ES,28.0,1,32,N,...,False,False,False,False,False,False,False,False,False,False
1653624,1553541,2019-05-01,UKNOWN,1,04 - SINCATEGORIA,ES,49.0,0,54,N,...,False,False,False,False,False,False,False,False,False,False
1653625,1553559,2019-05-01,UKNOWN,1,04 - SINCATEGORIA,ES,8.0,1,43,N,...,False,False,False,False,False,False,False,False,False,False
1653626,1553565,2019-05-01,UKNOWN,0,04 - SINCATEGORIA,ES,45.0,1,68,N,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1653628 entries, 0 to 1653627
Columns: 317 entries, pk_cid to encoded_44 days 12:00:00
dtypes: bool(276), datetime64[ns](2), float64(16), int32(9), int64(5), object(9)
memory usage: 895.7+ MB


In [ ]:
features_df.update(features_df.select_dtypes(include=bool).astype(int))

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el DataFrame en conjunto de características (X) y variable objetivo (y)
X = features_df.drop(columns=[TARGET])  # Características
y = features_df[TARGET]  # Variable objetivo

# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dividir el conjunto de entrenamiento en entrenamiento y validación (75% entrenamiento, 25% validación)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Mostrar el tamaño de los conjuntos de datos resultantes
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Codificar las columnas categóricas
label_encoders = {}
for column in ['entry_channel', 'segment', 'country_id', 'deceased', 'family_product', 'Provincia', 'SalaryRange']:
    label_encoders[column] = LabelEncoder()
    df4[column] = label_encoders[column].fit_transform(df4[column])

# Eliminar columnas no utilizables
df4.drop(columns=['pk_partition'], inplace=True)

# Dividir los datos en características (X) y variable objetivo (y)
X = df4.drop(columns=['product_desc'])
y = df4['product_desc']

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dividir el conjunto de entrenamiento en entrenamiento y validación (75% entrenamiento, 25% validación)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Mostrar el tamaño de los conjuntos de datos resultantes
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Convertir la variable objetivo a valores numéricos enteros
#label_encoder = LabelEncoder()
#y_train_encoded = label_encoder.fit_transform(y_train)
#y_val_encoded = label_encoder.transform(y_val)
#y_test_encoded = label_encoder.transform(y_test)

# Entrenar un modelo XGBoost con la variable objetivo codificada
model = xgb.XGBClassifier(objective='multi:softmax', num_class=len(label_encoder.classes_))
model.fit(X_train, y_train_encoded)

# Evaluar el modelo en el conjunto de validación
y_pred_val = model.predict(X_val)
accuracy_val = accuracy_score(y_val_encoded, y_pred_val)
print("Accuracy en el conjunto de validación:", accuracy_val)

# Evaluar el modelo en el conjunto de prueba
y_pred_test = model.predict(X_test)
accuracy_test = accuracy_score(y_test_encoded, y_pred_test)
print("Accuracy en el conjunto de prueba:", accuracy_test)